In [ ]:
import numpy as np
from scipy import optimize
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import sklearn as sk

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import utility_functions as fn

## Initialization

In [ ]:
# read from csv into a new dataframe
raw_weather = pd.read_csv('data/weather_raw2.csv',index_col=1)

# reindex by appropriate datetime
raw_weather.index = pd.to_datetime(raw_weather.index,format='%Y-%m-%d %H:00:00 +0000 UTC')
raw_weather.index.names = ['']

# remove unused columns
unused = raw_weather.columns[np.array([0,1,2,3,4,9,10,13,15,16,17,19,20,21,23,24,25,26])]
raw_weather.drop(unused,axis=1,inplace=True)

# rename and reorder columns
column_names = ['temp','tmin','tmax','pressure','humidity','wind','rain','snow','clouds']
raw_weather.columns = column_names
raw_weather = raw_weather[['temp','tmin','tmax','pressure','humidity','wind','clouds','rain','snow']]

In [ ]:
# drop duplicate rows
raw_weather.drop_duplicates(inplace=True)

# reindex to add missing rows, for a total length of 53545
correct_dt = pd.DatetimeIndex(start='2013-01-01 00:00:00',end='2019-02-10 00:00:00',freq='h')
raw_weather = raw_weather.reindex(index=correct_dt)

In [ ]:
print(raw_weather.isnull().sum())
# fn.plot_all(raw_weather,'2013-01-02 00:00:00','2019-02-09 00:00:00')

## Error Identification

In [ ]:
# set violating values to NaN based on hardcoded low and high limit values for each column separately
sparse_weather = raw_weather.copy(deep=True)

low_tuners = [250,240,250,900,5,0,0,0,0]
high_tuners = [350,350,350,1100,100,25,100,100,10]
j=0

for i in raw_weather.columns:
    sparse_weather[i].where(sparse_weather[i] >= low_tuners[j],inplace=True)
    sparse_weather[i].where(sparse_weather[i] <= high_tuners[j],inplace=True)
    j=j+1

In [ ]:
# assign NaN precip values to zero, this is an unconfirmed assumption
sparse_weather['rain'].replace(np.NaN,0,inplace=True)
sparse_weather['snow'].replace(np.NaN,0,inplace=True)

# set initial humidity value to suppress errors
sparse_weather['humidity'][0] = 46

In [ ]:
fn.plot_all(sparse_weather,'2013-01-01 00:00:00','2019-02-10 00:00:00')
# fn.plot_feature(sparse_weather,'temp','2016-07-16 00:00:00','2016-07-24 00:00:00',ylabel='Temp [K]')
# print(sparse_weather.isnull().sum())

## Imputation

In [ ]:
# impute missing ranges shorter than 6 hours using linear interpolation
dense_weather = sparse_weather.copy(deep=True)

# interpolate gaps in consumption data 6 hours and shorter
for k in dense_weather.columns:
    dense_weather[k] = fn.limited_impute(dense_weather[k],6)

fn.plot_feature(dense_weather,'temp','2016-07-16 00:00:00','2016-07-24 00:00:00',ylabel='Temp [K]')
print(dense_weather.isnull().sum())

In [ ]:
# implement sine_impute function, newly incompatible with fn.gaps for various reasons

# for now, linear impute over all gaps
dense_weather.interpolate(method='linear',inplace=True)

fn.plot_feature(dense_weather,'temp','2016-07-16 00:00:00','2016-07-24 00:00:00',ylabel='Temp [K]')
dense_weather.isnull().sum()

## Export

In [ ]:
dense_weather.to_csv('data/weather_clean.csv')

### unused code below

In [ ]:
# # returns index labels where nan values appear for a certain column
# nan_index = weather['temp'].index[weather['temp'].apply(np.isnan)]
# nan_index

# correct and complete datetime index for the date range considered
# dt = pd.DatetimeIndex(start='2013-01-01 00:00:00',end='2019-02-10 00:00:00',freq='h')

# # renames (here, makes lowercase) column labels using a simple loop
# df.columns = [x.lower() for x in df.columns]

# # implements sklearn scaler
# from sklearn import preprocessing
# x = df.values #returns a numpy array
# scaler = preprocessing.MinMaxScaler()
# x_scaled = scaler.fit_transform(x)
# df = pandas.DataFrame(x_scaled)

In [ ]:
# code from when i was adding an 'impute_ok' column

# TEMPORARY bypass missing 2014-2015 data
# weather = raw_weather.truncate('2016-01-01 00:00:00','2019-02-10 00:00:00')
# weather = raw_weather.copy(deep=False)

# add an 'impute_ok' column; True if time gap is less than 6 hours
# orig_len = len(raw_weather.index)
# raw_weather.insert(0,'impute_ok',True)
# raw_weather['impute_ok'].iloc[1:orig_len]=(raw_weather.index[1:orig_len]-raw_weather.index[0:orig_len-1] <= '06:00:00')

In [ ]:
# COULD BE USEFUL, first implementation of fit_sin that worked pretty well

# # assigning gap indices manually for now, use impute_ok later on
# gap_start = dense_weather.index.get_loc('2016-07-19 00:00:00')
# gap_end = dense_weather.index.get_loc('2016-07-20 17:00:00')
# gap_length = gap_end - gap_start

# # could calculate this as gap_start - (gap_length*2,3)
# impute_start = dense_weather.index.get_loc('2016-07-10 00:00:00')

# # subset dense_weather to the values just before the gap
# impute_data = dense_weather.iloc[impute_start:gap_start]
# impute_data.index = np.arange(len(impute_data))

# fit = fn.fit_sin(impute_data.index,impute_data['temp'])

# dense_weather.iloc[gap_start:gap_end,[0]] = pd.DataFrame(fit['fitfunc'](np.arange(len(impute_data),len(impute_data)+gap_length,1))).values

# fn.plot_feature(dense_weather,'temp','2016-07-16 00:00:00','2016-07-24 00:00:00',ylabel='Temp [K]')

# impute_data['temp'].values
# impute_data.iloc[:,[0]].T.values


In [ ]:
# FIRST PASS at writing sine_impute function, more updated version in utility_functions.py
# use fn.gaps_as_df AND fn.sine_fit to sine fit one column

# df_in = dense_weather.copy(deep=True)
# k = 0

# df_out = df_in.copy(deep=True)
# gaps = fn.gaps_as_df(bool_gaps,)



# for k in range(len(df_in.columns)):
    
# def sine_impute(df_in,column):
#     """
#     Imputes all gaps in data using a least-squares optimized sinusoidal fit function
    
#     df_in: DataFrame to which the target column belongs
#     column: str name of column to be imputed
#     """
#     df_out = df_in.copy(deep=True)
#     gaps = fn.gaps(df_in,column,6)

#     for i in gaps.index:

#         data_start_index = gaps.start[i] - 2*gaps.length[i]-1
#         data_end_index = gaps.start[i]-1
#         impute_start_index = gaps.start[i]
#         impute_end_index = gaps.end[i]

#         # print(gaps.start[i])
#         # print(gaps.end[i])

#         x_start = 0
#         x_end = data_end_index - data_start_index
#         x_impute_start = x_end+1
#         x_impute_end = impute_end_index - data_start_index

#         sine_fit = fn.sine_fit(df_in,df_in.columns[k],data_start_index,data_end_index)

#         imputed_values = sine_fit['fit'](np.arange(x_impute_start,x_impute_end))

#         # print(imputed_values)

#         df_in.iloc[gaps.start[i]:gaps.end[i],[k]] = pd.DataFrame(imputed_values).values
        
#     return df_out
